In [29]:
import ipywidgets as widgets
text_input = widgets.Text()
text_input.style.description_width = 'initial'  # Set description width to initial, allowing longer descriptions
text_input.description = 'Enter the model for feature extraction: '

# Display the widget
display(text_input)

feature_model = None

# Define a function to handle the input
def handle_submit(sender):
    global feature_model
    feature_model = text_input.value

# Set the function to handle input submission
text_input.on_submit(handle_submit)

Text(value='', description='Enter the model for feature extraction: ', style=TextStyle(description_width='init…

/var/folders/9d/b0mdn8zd7msdn25jg6z_ln3c0000gn/T/ipykernel_7994/3577263131.py:17: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(handle_submit)


In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from pickle import dump
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.applications import ResNet50, VGG16

def create_custom_cnn_model(input_shape):
    model = keras.Sequential([
        # Convolutional layers...
        keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        # Output layer with 4096 units
        keras.layers.Dense(4096)
    ])
    
    return model

def feature_extractions_cnn(directory):
    """
    Input: directory of images
    Return: A dictionary of features extracted by the custom CNN model.
    """
    input_shape = (224, 224, 3)  # Assuming input images are resized to 224x224
    num_classes = 4096  # Output feature size
    
    # Create and compile the custom CNN model
    input_shape = (224, 224, 3)
    model = create_custom_cnn_model(input_shape)
    model = keras.models.Model(inputs=model.input, outputs=model.layers[-1].output) 
# custom_cnn_model = create_custom_cnn_model(input_shape)
    model.compile(optimizer='adam', loss='mse')  # Adjust loss and optimizer as needed
    features = {}
    for f in os.listdir(directory):
        filename = os.path.join(directory, f)
        identifier = f.split('.')[0]
        
        # Load and preprocess image
        image = keras.preprocessing.image.load_img(filename, target_size=(input_shape[0], input_shape[1]))
        arr = keras.preprocessing.image.img_to_array(image)
        arr = np.expand_dims(arr, axis=0)
        arr = arr / 255.0  # Normalize pixel values
        
        # Extract feature
        feature = model.predict(arr)
        features[identifier] = feature
        
    
    return features


#Resnet 50 Model


def feature_extractions_resnet(directory):
    """
    Input: directory of images
    Return: A dictionary of features extracted by VGG-16, size 4096.
    """

    # Load ResNet50 model pre-trained on ImageNet data
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

    # Add a Dense layer with 4096 units
    x = base_model.output
    x = keras.layers.Dense(4096, activation='relu')(x)

    # Create a new model with the modified top
    model = keras.models.Model(inputs=base_model.input, outputs=x)
    
    features = {}
    for f in os.listdir(directory):
        filename = directory + "/" + f
        identifier = f.split('.')[0]
        
        image = keras.preprocessing.image.load_img(filename, target_size=(224,224))
        arr = keras.preprocessing.image.img_to_array(image, dtype=np.float32)
        arr = arr.reshape((1, arr.shape[0], arr.shape[1], arr.shape[2]))
        arr = keras.applications.resnet50.preprocess_input(arr)
    
        feature = model.predict(arr, verbose=0)
        features[identifier] = feature
    return features 




#VGG 50 Model
def feature_extractions_vgg16(directory):
    """
    Input: directory of images
    Return: A dictionary of features extracted by VGG-16, size 4096.
    """

    # Load ResNet50 model pre-trained on ImageNet data
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, pooling='avg')

    # Add a Dense layer with 4096 units
    x = base_model.output
    x = keras.layers.Dense(4096, activation='relu')(x)

    # Create a new model with the modified top
    model = keras.models.Model(inputs=base_model.input, outputs=x)
    
    features = {}
    for f in os.listdir(directory):
        filename = directory + "/" + f
        identifier = f.split('.')[0]
        
        image = keras.preprocessing.image.load_img(filename, target_size=(224,224))
        arr = keras.preprocessing.image.img_to_array(image, dtype=np.float32)
        arr = arr.reshape((1, arr.shape[0], arr.shape[1], arr.shape[2]))
        arr = keras.applications.vgg16.preprocess_input(arr)
    
        feature = model.predict(arr, verbose=0)
        features[identifier] = feature
    return features


In [ ]:
if feature_model.lower() == "custom":
    print("Ruuning the custom feature file")
    cnn_features = feature_extractions_cnn("Flicker8k_Dataset")
    with open("cnn_features.pkl", "wb") as file:
        pickle.dump(cnn_features, file)
        
if feature_model.lower() == 'resnet50':
    print("Ruuning the resnet feature file")
    resnet_features = feature_extractions_resnet("Flicker8k_Dataset")
    with open("resnet_features.pkl", "wb") as file:
        pickle.dump(resnet_features, file)
        
else:
    print("Ruuning the vgg16 feature file")
    vgg16_features = feature_extractions_vgg16("Flicker8k_Dataset")
    with open("vgg16_features.pkl", "wb") as file:
        pickle.dump(vgg16_features, file)



In [19]:
#testing of the code
len(vgg16_features['1000268201_693b08cb0e'][0])

4096